In [1]:
import json
from datasets import load_dataset, Dataset

# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [2]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)

In [3]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [4]:
import pickle
import pyarrow as pa
import pyarrow.parquet as pq

dataset_for_eval_df.to_csv('datasets/squad_processed.csv', index=False)

table = pa.Table.from_pandas(dataset_for_eval_df)
pq.write_table(table, 'datasets/squad_processed.parquet')


## ms_marco

In [2]:
from datasets import load_dataset

marco_ds = load_dataset("microsoft/ms_marco", "v1.1")
data_test = marco_ds['test']
ms_marco = data_test.select_columns(['passages', 'query', 'query_id'])
data_test

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 9650
})

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

msmarco_path = "datasets/msmarco_1000.parquet"
translated_marq_df = pd.read_parquet(msmarco_path, engine='pyarrow')

In [4]:
translated_marq_df

,id,query,passage_text
0,0,Da li ljudska kosa zaustavlja veverice?,"[Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit., Životinje ne podnose mir..."
1,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva je ta što se lako mogu skladištiti i transportovati sa jednog mesta na drugo. Velike rezerve uglja se stoga prenose iz rudnika uglja do industrijskih postrojenja koja su udaljena od rudnika. Nafta se takođe prenosi do udaljenih elektrana kako bi se pr...
2,2,šta je apotema,"[Apotema. Apotema pravilnog mnogougla je segment nacrtan od centra mnogougla (tj. centra kruga koji opisuje mnogougao) do jedne strane, tako da je normalan na stranu. Ako pravilni mnogougao ima površinu od A kvadratnih jedinica, obim od P jedinica, apotemu od a jedinica, tada je površina jednaka..."
3,3,prosečna cena za prilagođeni baldahin,"[Čestitamo! Pronašli ste BuyShade.com, vaš izvor za šatore koji se brzo postavljaju, prekrivače za stolove, prilagođene šatore, štampane zastave i još mnogo toga. Sa stotinama konfiguracija proizvoda i stilova u našem portfoliju i timom vrhunskih dizajnera, specijalizovani smo za pravljenje pril..."
4,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput tvrdog diska, koji je fizički povezan sa računarom ili nešto što se može fizički dodirnuti. CD-ROM, monitor računarskog ekrana, štampač i grafička kartica su primeri računarskog hardvera. Hardver služi kao sistem za isporuku softverskih rešenja. Har..."
...,...,...,...
959,994,koliko novca zarađuje umetnički direktor animatora u plati,"[Plata direktora animacije. Prema Zavodu za radnu statistiku, medijana godišnje zarade za direktore animacije (producente i reditelje) iznosi 71.350 dolara. Najnižih 10 procenata prosečno zarađuju manje od 32.080 dolara godišnje, dok najviših 10 procenata prosečno zarađuju više od 187.200 dolara..."
960,995,Da li je severni deo Tonga rova konvergentna pločna granica?,"[Tonga rov je konvergentna pločna granica. Rov leži na severnom kraju Kermadec-Tonga zone subdukcije, aktivne zone subdukcije gde se Pacifička ploča subdukuje ispod Tonga ploče i Indo-Australijske ploče. Takvi okeanski rovovi su važna mesta za formiranje onoga što će postati kontinentalna kora i..."
961,996,Da li sve infekcije desni dovode do apscesa?,"[Dentalni apsces je infekcija u ustima, licu, vilici ili grlu koja počinje kao infekcija zuba. Originalni uzrok može biti duboka rupa, parodontalna (desni) bolest ili trauma. Takve infekcije su verovatnije uzrokovane lošim zdravljem zuba i mogu rezultirati nedostatkom odgovarajuće i pravovremene..."
962,997,šta je imenovana osoba,"[Preciznije, ime je oznaka za određenu osobu, mesto ili stvar. Ponekad se nazivaju pravim imenima. Ostatak ovog članka bavi se imenima za ljude (lična imena), ali pogledajte imena mesta i imena brendova za druge vrste pravih imena. Lično ime je ime koje pripada osobi. Prezimena i data imena su p..."


In [6]:
id_dict = {row['id']: {'passage_text': row['passage_text'], 'query': row['query']} for _, row in translated_marq_df.iterrows()}
# id_dict

{'0': {'passage_text': array(['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
         'Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.',
         'Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će oterati veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.',
         '1 Možete posuti krvavu brašnu oko vaše bašte takođe. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno visoka. 4 Takođe, ako je živo

In [3]:

# # Calculate the length of each query
# query_lengths = [len(x) for x in data_test['query']]

# # Find the longest, shortest, and average query lengths
# longest_query = max(query_lengths)
# shortest_query = min(query_lengths)
# average_query_length = sum(query_lengths) / len(query_lengths)

# print(f"Longest query length: {longest_query}")
# print(f"Shortest query length: {shortest_query}")
# print(f"Average query length: {average_query_length:.2f}")

# print(len("sta je to ascci"))

In [17]:
titles = []
queries = []
contexts = []
for index, example in enumerate(data_test):
    # Access individual fields like 'query', 'query_id', and 'passages'
    query_id = str(example['query_id'])
    if query_id in id_dict:
        query = id_dict[query_id]['query']
        is_selected = example['passages']['is_selected']
        passage_text = id_dict[query_id]['passage_text']
        if len(passage_text) != len(is_selected):
            print(f"{len(passage_text)}____{len(is_selected)}")
        for i in range(min(len(is_selected), len(passage_text))):
            titles.append("")
            if is_selected[i] == 1:
                queries.append([query])
                contexts.append(passage_text[i])
            else:
                queries.append([])
                contexts.append(passage_text[i])

print()


11____9
11____9
8____9
7____8
7____8
9____10
8____9
7____8
8____9
7____9
9____10
10____9
5____9
9____10
9____10
8____10
7____8
9____10
9____10
6____10
8____9
8____9
8____9
6____7
8____9
8____9
6____7
4____9
4____7
8____7
4____7
8____9
8____9
11____10
7____8
8____9
7____8
9____10
7____9
7____9
9____10
12____8
8____9
8____9
7____9
8____6
8____10
8____10
7____8
7____8
10____6
7____9
8____9
6____8
8____9
7____10
12____10
6____9
5____6
8____9
6____7
7____8
7____8
7____8
9____10
11____9
4____5
5____6
8____9
10____8
8____9
7____10
10____9
8____9
6____7
8____9
8____9
8____9
7____8
5____10
15____8
9____10
8____9
8____9
8____9
7____9
10____9
9____10
9____10



In [15]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queries})

dataset_for_eval_df.head()

,title,context,queries
0,,"Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.",[]
1,,"Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.",[]
2,,Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će oterati veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.,[Da li ljudska kosa zaustavlja veverice?]
3,,"1 Možete posuti krvavu brašnu oko vaše bašte takođe. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno visoka. 4 Takođe, ako je životinja ženka, postoji velika verovatnoća da ćete je odvojiti od beba koje zavise od nje za preživljavanje.",[]
4,,Gubitak dlake kod veverica može biti uzrokovan šugaravicom ili gljivičnom bolešću. Šugaravica je bolest izazvana mikroskopskim grinjama koje se uvlače ispod kože. Grinja šugaravice kod veverica je zabeležena kako kod lisica tako i kod sivih veverica. Nema izveštaja da je ovaj tip šugaravice pren...,[]


In [16]:
len(dataset_for_eval_df)

7842

In [9]:
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/marco_processed.parquet')